# Expression remapper

This notebook is a simple app that allows users to map different facial expressions and motions to keyboard inputs. I aim to use this to play a video game like Dark Souls as a test.

Using MediaPipe, and following their [face landmark detection guide](https://developers.google.com/mediapipe/solutions/vision/face_landmarker/python).

## Install dependencies

#### Mediapipe

In [ ]:
!pip install mediapipe
# opencv as well

## Import dependencies

In [1]:
# Mediapipe for face landmark detection
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# OpenCV for drawing utilities and webcam input
import cv2

## Load the model

In [2]:
model_path = 'models/face_landmarker.task'

## Create the Face Landmarker task

In [21]:
import mediapipe as mp
import math

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
FaceLandmarkerResult = mp.tasks.vision.FaceLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

# # Global variables to store the width, height, and channels of the webcam image
# img_w = 0;
# img_h = 0;
# img_c = 0;

yaw = 0 # Rotate left/right
pitch = 0 # Tilt up/down
roll = 0 # Tilt left/right

# Create a face landmarker instance with the live stream mode:
def print_result(result: FaceLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    #print('face landmarker result: {}'.format(result))
    #print('facial transformation matrixes: {}'.format(result.facial_transformation_matrixes))
    
    face_3d = []
    face_2d = []
    
    # Below code for calculating the orientation of the head has been adapted from Nico Nielsen's code
    # https://github.com/niconielsen32/ComputerVision/blob/master/headPoseEstimation.py
    # If any landmarks have been detected..
    if result.face_landmarks:
        mtrix = result.facial_transformation_matrixes[0]
        global yaw
        global pitch
        global roll
                
        # Get the yaw pitch and roll from the transformation matrix (yuck.. I still barely understand these darn things :/)
        if mtrix[0][0] == 1.0:
            yaw = math.atan2(mtrix[0][2], mtrix[2][3])
            pitch = 0
            roll = 0
        
        elif mtrix[0][0] == -1.0:
            yaw = math.atan2(mtrix[0][0], mtrix[2][3])
            pitch = 0
            roll = 0
        
        else:
            yaw = math.atan2(-mtrix[2][0], mtrix[0][0])
            pitch = math.atan2(-mtrix[1][2], mtrix[1][1])
            roll = math.asin(mtrix[1][0])
            
        
        # CALIBRATE THESE VALUES TO COMPARE BY ASKING USER TO LOOK DIFFERENT WAYS
#         # Yaw (look left/right)
#         print("yaw is: {}".format(math.degrees(yaw)))
#         if yaw > 0:
#             print("looking left")
#         elif yaw < 0:
#             print("looking right")
        
#         # Pitch (look up/down)
#         print("pitch is: {}".format(math.degrees(pitch)))
#         if yaw > 0:
#             print("looking up")
#         elif yaw < 0:
#             print("looking down")
        
        # Roll (rilt left/right)
        print("roll is: {}".format(math.degrees(roll)))
        if roll > 0:
            print("looking rolled right")
        elif roll < 0:
            print("looking rolled left")
            
            
            #print(math.degrees(pitch)) # pitch is actually roll? (left/right)
            #print(math.degrees(roll)) # roll is actually pitch (up/down)
            #print(math.degrees(yaw)) # yaw is actually yaw
                
    

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result,
    output_facial_transformation_matrixes=True, ## TRY CHANGING THIS
    output_face_blendshapes=False) ## TRY CHANGING THIS

with FaceLandmarker.create_from_options(options) as landmarker:
    # The landmarker is initialized. Use it here.
    # Use OpenCV’s VideoCapture to start capturing from the webcam.
    video = cv2.VideoCapture(0)
    
    # Retrieve and store image width, height, and channels globally for use later
    ret, frame = video.read()
    global img_w, img_h, img_c;
    img_h, img_w, img_c = frame.shape
    
    # Initialise the frame timestamp (MAKE THIS IN ms?)
    frame_timestamp = 0
    
    # Create a loop to read the latest frame from the camera using VideoCapture#read()
    while video.isOpened():
        ret, frame = video.read()
        
        # Convert the frame received from OpenCV to a MediaPipe Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        
        # Send live image data to perform face landmarking.
        # The results are accessible via the `result_callback` provided in
        # the `PoseLandmarkerOptions` object.
        # The pose landmarker must be created with the live stream mode.
        landmarker.detect_async(mp_image, frame_timestamp)
        
        # Increment the timestamp
        frame_timestamp += 1
        
        # Draw the output
        cv2.imshow('window', frame)
        
        # Break out of the loop
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Clean up
video.release()
cv2.destroyAllWindows()

roll is: 4.542201623803617
looking rolled right
roll is: 4.504538334593013
looking rolled right
roll is: 4.501497625198728
looking rolled right
roll is: 4.484877360188209
looking rolled right
roll is: 4.44597361092662
looking rolled right
roll is: 4.405909909784767
looking rolled right
roll is: 4.356297295128932
looking rolled right
roll is: 4.3794561465789394
looking rolled right
roll is: 4.426419106011769
looking rolled right
roll is: 4.45639803952565
looking rolled right
roll is: 4.5252023068451495
looking rolled right
roll is: 4.577105501947275
looking rolled right
roll is: 4.571295422673418
looking rolled right
roll is: 4.519145564339244
looking rolled right
roll is: 4.47791832003978
looking rolled right
roll is: 4.338283778545325
looking rolled right
roll is: 4.1887348409408105
looking rolled right
roll is: 4.064056925756866
looking rolled right
roll is: 3.406378171958549
looking rolled right
roll is: 2.7048527870961965
looking rolled right
roll is: 1.5881652498182646
looking rol